<a href="https://colab.research.google.com/github/r2prof/Optimization/blob/main/supply_network_design/supply_network_design_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supply Network Design 1

## Objective and Prerequisites

Try this Jupyter Notebook Modeling Example to learn how to solve a classic supply network design problem that involves finding the minimum cost flow through a network. We’ll show you how – given a set of factories, depots, and customers – you can use mathematical optimization to determine the best way to satisfy customer demand while minimizing shipping costs.

This model is example 19 from the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams on pages 273-275 and 330-332.

This example is of beginning difficulty; we assume that you know Python and have some knowledge of the Gurobi Python API and building mathematical optimization models.

**Download the Repository** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip).

---
## Problem Description

In this problem, we have six end customers, each with a known demand for a product.  Customer demand can be satisfied from a set of four depots, or directly from a set of two factories.  Each depot can support a maximum volume of product moving through it, and each factory can produce a maximum amount of product.  There are known costs associated with transporting the product, from a factory to a depot, from a depot to a customer, or from a factory directly to a customer.

Our supply network has two factories, in Liverpool and Brighton, that produce a product.  Each has a maximum production capacity:

| Factory | Supply (tons) |
| --- | --- |
| Liverpool | 150,000 |
| Brighton |  200,000 |

The product can be shipped from a factory to a set of four depots.  Each depot has a maximum throughput.  Depots don't produce or consume the product; they simply pass the product on to customers.

| Depot | Throughput (tons) |
| --- | --- |
| Newcastle | 70,000 |
| Birmingham | 50,000 |
| London | 100,000 |
| Exeter | 40,000 |

Our network has six customers, each with a given demand.

| Customer | Demand (tons) |
| --- | --- |
| C1 | 50,000 |
| C2 | 10,000 |
| C3 | 40,000 |
| C4 | 35,000 |
| C5 | 60,000 |
| C6 | 20,000 |

Shipping costs are given in the following table (in dollars per ton).  Columns are source cities and rows are destination cities.  Thus, for example, it costs $1 per ton to ship the product from Liverpool to London.  A '-' in the table indicates that that combination is not possible, so for example it is not possible to ship from the factory in Brighton to the depot in Newcastle.

| To | Liverpool | Brighton | Newcastle | Birmingham | London | Exeter |
| --- | --- | --- | --- | --- | --- | --- |
| Depots |
| Newcastle  | 0.5 |   - |
| Birmingham | 0.5 | 0.3 |
| London     | 1.0 | 0.5 |
| Exeter     | 0.2 | 0.2 |
| Customers |
| C1 | 1.0 | 2.0 |   - | 1.0 |   - |   - |
| C2 |   - |   - | 1.5 | 0.5 | 1.5 |   - |
| C3 | 1.5 |   - | 0.5 | 0.5 | 2.0 | 0.2 |
| C4 | 2.0 |   - | 1.5 | 1.0 |   - | 1.5 |
| C5 |   - |   - |   - | 0.5 | 0.5 | 0.5 |
| C6 | 1.0 |   - | 1.0 |   - | 1.5 | 1.5 |

The question to be answered is how to satisfy the demands of the end customers while minimizing shipping costs.

---
## Model Formulation

### Sets and Indices

$f \in \text{Factories}=\{\text{Liverpool}, \text{Brighton}\}$

$d \in \text{Depots}=\{\text{Newcastle}, \text{Birmingham}, \text{London}, \text{Exeter}\}$

$c \in \text{Customers}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{Cities} = \text{Factories} \cup \text{Depots} \cup \text{Customers}$

### Parameters

$\text{cost} \in \mathbb{R}^+$: Cost of shipping one ton from source $s$ to destination $t$.

$\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from factory $f$ (in tons).

$\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in tons).

$\text{demand}_c \in \mathbb{R}^+$: Demand for goods at customer $c$ (in tons).

### Decision Variables

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of goods (in tons) that is shipped from source $s$ to destionation $t$.


### Objective Function

- **Cost**: Minimize total shipping costs.

\begin{equation}
\text{Minimize} \quad Z = \sum_{f,d}{\text{c}_{fd}\text{x}_{fd}}+
                           \sum_{f,c}{\text{d}_{fc}\text{y}_{fc}}+
                           \sum_{d,c}{\text{e}_{dc}\text{z}_{dc}}
\end{equation}

### Constraints

- **Factory capacities**: 

\begin{equation}
\sum_{d \in \text{Depots}}{\text{x}_{fd}}+\sum_{c \in \text{Customers}}{\text{y}_{fc}}\leq \text{C}_{f}\quad \forall f \in \text{Factories}
\end{equation}

- **Quantity into Depots**: 

\begin{equation}
\sum_{f \in \text{Factories}}{\text{x}_{fd}} \leq \text{C}_{d} \quad \forall d \in \text{Depots}
\end{equation}

- **Quantity out of Depots**: 

\begin{equation}
\sum_{f \in \text{Factories}}{\text{x}_{fd}} =
\sum_{c \in \text{Customers}}{\text{z}_{dc}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Customer Demand**: 

\begin{equation}
\sum_{f \in \text{Factories}}{\text{y}_{fc}} + \sum_{d \in \text{Depots}}{\text{z}_{dc}}= \text{D}_{c}
\quad \forall c \in \text{Customers}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

![](SC-Network-I.png)

In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Sets Factories (F), Depots (D), and Customers (C)
# When we code sets we can be more descriptive in the name
factories = ['Liverpool', 'Brighton']
depots = ['Newcastle', 'Birmingham', 'London', 'Exeter']
customers = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']

In [3]:
# Define a gurobipy model for the decision problem
m = gp.Model('SCND-I')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-29


## Parameters

Parameters of a math optimization problem are values treated as constants in the model and are associated with the decision variables. For this decision problem these values are the limits of each production facility, the demand for each distribution center, and the pairwise costs between production and distribution locations.

- $C_f$ is the max production capcity (in tons) of a factory at location $f$, $\forall f \in F \quad\quad\quad\quad\quad\quad\quad\quad\quad\space\space \texttt{fatory}\_\texttt{capacity[f]}$
- $C_d$ is the maximum possible flow through depot capcity (in tons) in location $d$, $\forall d \in D \quad\quad\quad\quad\quad\quad \texttt{depot}\_\texttt{capacity[d]}$
- $D_c$ is the demand for goods (in tons) by the customer in location $c$, $\forall c \in C
\quad\quad\quad\quad\quad\quad\quad\quad\quad\space\space \texttt{customer}\_\texttt{demand[c]}$
- $c_{f,d}$ is the cost to ship between location $f$ and location $d$, $\forall f \in F, d \in D \quad\quad\quad \texttt{cost[f,d]}$
- $d_{f,c}$ is the cost to ship between location $f$ and location $c$, $\forall f \in F, c \in C \quad\quad\quad \texttt{cost[f,c]}$
- $e_{d,c}$ is the cost to ship between location $d$ and location $c$, $\forall d \in D, c \in C \quad\quad\quad \texttt{cost[d,c]}$

In [4]:
factory_capacity = pd.Series([150000, 200000], index = factories, name = "factory_capacity")
factory_capacity.to_frame()
#factory_capacity

,factory_capacity
Liverpool,150000
Brighton,200000


In [5]:
depot_capcity = pd.Series([70000, 50000, 100000, 40000], index = depots, name = "depot_capacity")
depot_capcity.to_frame()
#depot_capcity

,depot_capacity
Newcastle,70000
Birmingham,50000
London,100000
Exeter,40000


In [6]:
customer_demand = pd.Series([50000,10000,40000,35000,60000,20000], index = customers, name = 'customer_demand')
customer_demand.to_frame()
#customer_demand 

,customer_demand
C1,50000
C2,10000
C3,40000
C4,35000
C5,60000
C6,20000


In [7]:
# Read transportation cost data between factory and depot
data = pd.read_csv('../fd_cost.csv', index_col=[0,1])

# Use squeeze=True to make the costs a series
trans_fd_cost = data.squeeze()
trans_fd_cost

factories  depots    
Liverpool  Newcastle        0.5
           Birmingham       0.5
           London           1.0
           Exeter           0.2
Brighton   Newcastle     1000.0
           Birmingham       0.3
           London           0.5
           Exeter           0.2
Name: cost, dtype: float64

In [8]:
# Read transportation cost data between factory and customer
data = pd.read_csv('../fc_cost.csv', index_col=[0,1])

# Use squeeze=True to make the costs a series
trans_fc_cost = data.squeeze()
trans_fc_cost

factories  customers
Liverpool  C1              1.0
           C2           1000.0
           C3              1.5
           C4              2.0
           C5           1000.0
           C6              1.0
Brighton   C1              2.0
           C2           1000.0
           C3           1000.0
           C4           1000.0
           C5           1000.0
           C6           1000.0
Name: cost, dtype: float64

In [9]:
# Read transportation cost data between depot and customer
data = pd.read_csv('../dc_cost.csv', index_col=[0,1])

# Use squeeze=True to make the costs a series
trans_dc_cost = data.squeeze()
trans_dc_cost

depots      customers
Newcastle   C1           1000.0
            C2              1.5
            C3              0.5
            C4              1.5
            C5           1000.0
            C6              1.0
Birmingham  C1              1.0
            C2              0.5
            C3              0.5
            C4              1.0
            C5              0.5
            C6           1000.0
London      C1           1000.0
            C2              1.5
            C3              2.0
            C4           1000.0
            C5              0.5
            C6              1.5
Exeter      C1           1000.0
            C2           1000.0
            C3              0.2
            C4              1.5
            C5              0.5
            C6              1.5
Name: cost, dtype: float64

## Input Data
We define all the input data for the model.

## Decision Variables
$x_{f,d} =$ quantity sent from factory $f$ to depot $d$ $\qquad\qquad$ $f=1,2, \qquad d=1,2,3,4$

$y_{f,c} =$ quantity sent from factory $f$ to customer $c$ $\qquad\qquad$ $f=1,2, \qquad c=1,2,3,4,5,6$

$z_{d,c} =$ quantity sent from depot $d$ to customer $c$ $\qquad\qquad$ $d=1,2,3,4, \qquad c=1,2,3,4,5,6$

In [10]:
# First method
# Loop through each f and d combination to create a decision variable
x = {}
for f in factories:
    for d in depots:
        x[f,d] = m.addVar(name = f+"_to_"+d)
m.update()
x

{('Liverpool', 'Newcastle'): <gurobi.Var Liverpool_to_Newcastle>,
 ('Liverpool', 'Birmingham'): <gurobi.Var Liverpool_to_Birmingham>,
 ('Liverpool', 'London'): <gurobi.Var Liverpool_to_London>,
 ('Liverpool', 'Exeter'): <gurobi.Var Liverpool_to_Exeter>,
 ('Brighton', 'Newcastle'): <gurobi.Var Brighton_to_Newcastle>,
 ('Brighton', 'Birmingham'): <gurobi.Var Brighton_to_Birmingham>,
 ('Brighton', 'London'): <gurobi.Var Brighton_to_London>,
 ('Brighton', 'Exeter'): <gurobi.Var Brighton_to_Exeter>}

$y_{f,c} =$ quantity sent from factory $f$ to customer $c$ $\qquad\qquad$ $f=1,2, \qquad c=1,2,3,4,5,6$

In [11]:
# Second method
# Provide each set for the indices
y = m.addVars(factories, customers, name = "factory_to_cust")
m.update()
y

{('Liverpool', 'C1'): <gurobi.Var factory_to_cust[Liverpool,C1]>,
 ('Liverpool', 'C2'): <gurobi.Var factory_to_cust[Liverpool,C2]>,
 ('Liverpool', 'C3'): <gurobi.Var factory_to_cust[Liverpool,C3]>,
 ('Liverpool', 'C4'): <gurobi.Var factory_to_cust[Liverpool,C4]>,
 ('Liverpool', 'C5'): <gurobi.Var factory_to_cust[Liverpool,C5]>,
 ('Liverpool', 'C6'): <gurobi.Var factory_to_cust[Liverpool,C6]>,
 ('Brighton', 'C1'): <gurobi.Var factory_to_cust[Brighton,C1]>,
 ('Brighton', 'C2'): <gurobi.Var factory_to_cust[Brighton,C2]>,
 ('Brighton', 'C3'): <gurobi.Var factory_to_cust[Brighton,C3]>,
 ('Brighton', 'C4'): <gurobi.Var factory_to_cust[Brighton,C4]>,
 ('Brighton', 'C5'): <gurobi.Var factory_to_cust[Brighton,C5]>,
 ('Brighton', 'C6'): <gurobi.Var factory_to_cust[Brighton,C6]>}

$z_{d,c} =$ quantity sent from depot $d$ to customer $c$ $\qquad\qquad$ $d=1,2,3,4, \qquad c=1,2,3,4,5,6$

In [12]:
# Third method
# The index of the tranporation costs have each combination of depot and customer location
z = m.addVars(trans_dc_cost.index, name = 'depot_to_cust')
m.update()
z

{('Newcastle', 'C1'): <gurobi.Var depot_to_cust[Newcastle,C1]>,
 ('Newcastle', 'C2'): <gurobi.Var depot_to_cust[Newcastle,C2]>,
 ('Newcastle', 'C3'): <gurobi.Var depot_to_cust[Newcastle,C3]>,
 ('Newcastle', 'C4'): <gurobi.Var depot_to_cust[Newcastle,C4]>,
 ('Newcastle', 'C5'): <gurobi.Var depot_to_cust[Newcastle,C5]>,
 ('Newcastle', 'C6'): <gurobi.Var depot_to_cust[Newcastle,C6]>,
 ('Birmingham', 'C1'): <gurobi.Var depot_to_cust[Birmingham,C1]>,
 ('Birmingham', 'C2'): <gurobi.Var depot_to_cust[Birmingham,C2]>,
 ('Birmingham', 'C3'): <gurobi.Var depot_to_cust[Birmingham,C3]>,
 ('Birmingham', 'C4'): <gurobi.Var depot_to_cust[Birmingham,C4]>,
 ('Birmingham', 'C5'): <gurobi.Var depot_to_cust[Birmingham,C5]>,
 ('Birmingham', 'C6'): <gurobi.Var depot_to_cust[Birmingham,C6]>,
 ('London', 'C1'): <gurobi.Var depot_to_cust[London,C1]>,
 ('London', 'C2'): <gurobi.Var depot_to_cust[London,C2]>,
 ('London', 'C3'): <gurobi.Var depot_to_cust[London,C3]>,
 ('London', 'C4'): <gurobi.Var depot_to_cust[L

## Constraints
We outlined production and demand constraints at the beginning of this example; now we formulate and code them. Note that it doesn't matter the order in which constraints (and/or decision variables) are added to the model.

- **Factory capacities**: 

\begin{equation}
\sum_{d \in \text{Depots}}{\text{x}_{fd}}+\sum_{c \in \text{Customers}}{\text{y}_{fc}}\leq \text{C}_{f}\quad \forall f \in \text{Factories}
\end{equation}

In [13]:
fac_cap = m.addConstrs((gp.quicksum(x[f,d] for d in depots) + gp.quicksum(y[f,c] for c in customers) <= factory_capacity[f] for f in factories), name = 'fac_cap')
m.update()
fac_cap

{'Liverpool': <gurobi.Constr fac_cap[Liverpool]>,
 'Brighton': <gurobi.Constr fac_cap[Brighton]>}

- **Quantity into Depots**: 

\begin{equation}
\sum_{f \in \text{Factories}}{\text{x}_{fd}} \leq \text{C}_{d} \quad \forall d \in \text{Depots}
\end{equation}

In [14]:
dep_cap = m.addConstrs((gp.quicksum(x[f,d] for f in factories) <= depot_capcity[d] for d in depots), name = 'dep_cap')
m.update()
dep_cap

{'Newcastle': <gurobi.Constr dep_cap[Newcastle]>,
 'Birmingham': <gurobi.Constr dep_cap[Birmingham]>,
 'London': <gurobi.Constr dep_cap[London]>,
 'Exeter': <gurobi.Constr dep_cap[Exeter]>}

- **Quantity out of Depots**: 

\begin{equation}
\sum_{f \in \text{Factories}}{\text{x}_{fd}} =
\sum_{c \in \text{Customers}}{\text{z}_{dc}}
\quad \forall d \in \text{Depots}
\end{equation}

In [15]:
dep_out = m.addConstrs((gp.quicksum(x[f,d] for f in factories) == gp.quicksum(z[d,c] for c in customers) for d in depots), name = 'dep_out')
m.update()
dep_out

{'Newcastle': <gurobi.Constr dep_out[Newcastle]>,
 'Birmingham': <gurobi.Constr dep_out[Birmingham]>,
 'London': <gurobi.Constr dep_out[London]>,
 'Exeter': <gurobi.Constr dep_out[Exeter]>}

- **Customer Demand**: 

\begin{equation}
\sum_{f \in \text{Factories}}{\text{y}_{fc}} + \sum_{d \in \text{Depots}}{\text{z}_{dc}}= \text{D}_{c}
\quad \forall c \in \text{Customers}
\end{equation}

In [16]:
cus_demand = m.addConstrs((gp.quicksum(y[f,c] for f in factories) + gp.quicksum(z[d,c] for d in depots) == customer_demand[c] for c in customers), name = 'cus_demand')
m.update()
cus_demand

{'C1': <gurobi.Constr cus_demand[C1]>,
 'C2': <gurobi.Constr cus_demand[C2]>,
 'C3': <gurobi.Constr cus_demand[C3]>,
 'C4': <gurobi.Constr cus_demand[C4]>,
 'C5': <gurobi.Constr cus_demand[C5]>,
 'C6': <gurobi.Constr cus_demand[C6]>}

### Objective Function

- **Cost**: Minimize total shipping costs.

\begin{equation}
\text{Minimize} \quad Z = \sum_{f,d}{\text{c}_{fd}\text{x}_{fd}}+
                           \sum_{f,c}{\text{d}_{fc}\text{y}_{fc}}+
                           \sum_{d,c}{\text{e}_{dc}\text{z}_{dc}}
\end{equation}

In [17]:
m.setObjective(gp.quicksum(trans_fd_cost[f,d]*x[f,d] for f in factories for d in depots)+gp.quicksum(trans_fc_cost[f,c]*y[f,c] for f in factories for c in customers)+gp.quicksum(trans_dc_cost[d,c]*z[d,c] for d in depots for c in customers), GRB.MINIMIZE)


## Find, Extract, and Analyze the Solution
Before running the optimization, it is a good idea to write an `lp` file. This is a text file that prints out the variables, constraints, and object like we would see in the *formulation*, just without the summation symbols and using the names we designated.

In [18]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Linux Mint 21.3")

CPU model: AMD Ryzen 5 5600G with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16 rows, 44 columns and 96 nonzeros
Model fingerprint: 0xe073b711
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve time: 0.00s
Presolved: 16 rows, 44 columns, 96 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4800000e+05   1.450000e+05   0.000000e+00      0s
       6    1.9850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.985000000e+05


In [19]:
# You can get the name and value of all the decision variables:
all_vars = {v.varName: v.x for v in m.getVars()}
all_vars

{'Liverpool_to_Newcastle': 0.0,
 'Liverpool_to_Birmingham': 0.0,
 'Liverpool_to_London': 0.0,
 'Liverpool_to_Exeter': 40000.0,
 'Brighton_to_Newcastle': 0.0,
 'Brighton_to_Birmingham': 50000.0,
 'Brighton_to_London': 55000.0,
 'Brighton_to_Exeter': 0.0,
 'factory_to_cust[Liverpool,C1]': 50000.0,
 'factory_to_cust[Liverpool,C2]': 0.0,
 'factory_to_cust[Liverpool,C3]': 0.0,
 'factory_to_cust[Liverpool,C4]': 0.0,
 'factory_to_cust[Liverpool,C5]': 0.0,
 'factory_to_cust[Liverpool,C6]': 20000.0,
 'factory_to_cust[Brighton,C1]': 0.0,
 'factory_to_cust[Brighton,C2]': 0.0,
 'factory_to_cust[Brighton,C3]': 0.0,
 'factory_to_cust[Brighton,C4]': 0.0,
 'factory_to_cust[Brighton,C5]': 0.0,
 'factory_to_cust[Brighton,C6]': 0.0,
 'depot_to_cust[Newcastle,C1]': 0.0,
 'depot_to_cust[Newcastle,C2]': 0.0,
 'depot_to_cust[Newcastle,C3]': 0.0,
 'depot_to_cust[Newcastle,C4]': 0.0,
 'depot_to_cust[Newcastle,C5]': 0.0,
 'depot_to_cust[Newcastle,C6]': 0.0,
 'depot_to_cust[Birmingham,C1]': 0.0,
 'depot_to_cust[

In [22]:
for v in m.getVars():
    print(f"{v.VarName} {v.X:g}")

print(f"Objective Function: {m.ObjVal:g}")

Liverpool_to_Newcastle 0
Liverpool_to_Birmingham 0
Liverpool_to_London 0
Liverpool_to_Exeter 40000
Brighton_to_Newcastle 0
Brighton_to_Birmingham 50000
Brighton_to_London 55000
Brighton_to_Exeter 0
factory_to_cust[Liverpool,C1] 50000
factory_to_cust[Liverpool,C2] 0
factory_to_cust[Liverpool,C3] 0
factory_to_cust[Liverpool,C4] 0
factory_to_cust[Liverpool,C5] 0
factory_to_cust[Liverpool,C6] 20000
factory_to_cust[Brighton,C1] 0
factory_to_cust[Brighton,C2] 0
factory_to_cust[Brighton,C3] 0
factory_to_cust[Brighton,C4] 0
factory_to_cust[Brighton,C5] 0
factory_to_cust[Brighton,C6] 0
depot_to_cust[Newcastle,C1] 0
depot_to_cust[Newcastle,C2] 0
depot_to_cust[Newcastle,C3] 0
depot_to_cust[Newcastle,C4] 0
depot_to_cust[Newcastle,C5] 0
depot_to_cust[Newcastle,C6] 0
depot_to_cust[Birmingham,C1] 0
depot_to_cust[Birmingham,C2] 10000
depot_to_cust[Birmingham,C3] 0
depot_to_cust[Birmingham,C4] 35000
depot_to_cust[Birmingham,C5] 5000
depot_to_cust[Birmingham,C6] 0
depot_to_cust[London,C1] 0
depot_to_cus

---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC